In [1]:
# =========================
# IMPORTS
# =========================

from X_functions import *
from X_config import *
from X_models import *
from X_interpolation import *
from X_products import *

from GGS_main import *

# =========================

c:\Users\sal_f\anaconda3\envs\ggs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

### GGS Executable

In [2]:
power=1
path="local"
config_name="ru29"

In [3]:
if config_name is None:
    print("No config file specified. Exiting.")
    exit(1)

config = GGS_config_import(config_name)

target_datetime = config['MISSION'].get('target_date')
if not target_datetime:
    print("Issue with target datetime. Using current datetime.")
    target_datetime = dt.datetime.now(dt.timezone.utc)

if path == "local":
    root_directory = GGS_config_process(config, path="default")
elif path == "rucool":
    root_directory = GGS_config_process(config, path="/www/web/rucool/hurricane/model_comparisons/maps/yucatan")
else:
    raise ValueError("Invalid root directory.")

datetime_list = [target_datetime.replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%dT%H:%M:%SZ')]
datetime_index = datetime_list[0]

glider_dataframes = None
if config['PRODUCT'].get('show_gliders'):
    min_lat, min_lon = config['MISSION']['extent'][0]
    max_lat, max_lon = config['MISSION']['extent'][1]
    search_extent = [min_lon, max_lon, min_lat, max_lat]
    glider_dataframes = acquire_gliders(
        extent=search_extent,
        target_date=target_datetime,
        date_delta=dt.timedelta(days=1),
        requested_variables=["time", "longitude", "latitude", "profile_id", "depth"],
        print_vars=False,
        target="all",
        request_timeout=5,
        enable_parallel=False
    )


### IMPORTING GGS CONFIGURATION: ru29 ###

Configuration import success!

### PROCESSING GGS CONFIGURATION ###

Configuration:

--> MISSION
mission_name: RU29
target_date: 2024-04-26T13:36:53+0000
max_depth: 1000
extent: ((12, -71), (19, -66))
GPS_coords: [[17.757, -67.659], [17.75, -69.0], [17.0, -69.0], [16.0, -69.0], [15.0, -69.0], [14.0, -69.0], [13.0, -69.0], [14.0, -69.0], [15.0, -69.0], [16.0, -69.0], [17.0, -69.0], [17.75, -69.0]]
glider_id: None
glider_buffer: 3

--> MODEL
single_datetime: True
enable_rtofs: True
enable_cmems: True
enable_gofs: True
chunk: True
save_model_data: False
save_depth_average: True
save_bin_average: False

--> PRODUCT
create_magnitude_plot: False
create_threshold_plot: False
create_advantage_plot: False
create_profile_plot: True
create_gpkg_file: False
latitude_qc: 17.757
longitude_qc: -67.659
density: 3
mag1: 0.0
mag2: 0.2
mag3: 0.3
mag4: 0.4
mag5: 0.5
tolerance: 15
show_gliders: True
show_waypoints: True
show_eez: True
show_qc: False
manual_extent

In [4]:
datetime_index = datetime_index
config_flag = config
root_directory_flag = root_directory
glider_data_flag = glider_dataframes

enable_rtofs_flag = config_flag['MODEL']['enable_rtofs']
enable_cmems_flag = config_flag['MODEL']['enable_cmems']
enable_gofs_flag = config_flag['MODEL']['enable_gofs']
save_model_data_flag = config_flag['MODEL']['save_model_data']
save_depth_average_flag = config_flag['MODEL']['save_depth_average']
save_bin_average_flag = config_flag['MODEL']['save_bin_average']
chunk_flag = config_flag['MODEL']['chunk']

create_magnitude_plot_flag = config_flag['PRODUCT']['create_magnitude_plot']
create_threshold_plot_flag = config_flag['PRODUCT']['create_threshold_plot']
create_advantage_plot_flag = config_flag['PRODUCT']['create_advantage_plot']
create_profiles_plot_flag = config_flag['PRODUCT']['create_profile_plot']
create_gpkg_file_flag = config_flag['PRODUCT']['create_gpkg_file']
latitude_qc_flag = config_flag['PRODUCT']['latitude_qc']
longitude_qc_flag = config_flag['PRODUCT']['longitude_qc']
density_flag = config_flag['PRODUCT']['density']
mag1_flag = config_flag['PRODUCT']['mag1']
mag2_flag = config_flag['PRODUCT']['mag2']
mag3_flag = config_flag['PRODUCT']['mag3']
mag4_flag = config_flag['PRODUCT']['mag4']
mag5_flag = config_flag['PRODUCT']['mag5']
tolerance_flag = config_flag['PRODUCT']['tolerance']
show_waypoints_flag = config_flag['PRODUCT']['show_waypoints']
show_eez_flag = config_flag['PRODUCT']['show_eez']
show_qc_flag = config_flag['PRODUCT']['show_qc']
manual_extent_flag = config_flag['PRODUCT']['manual_extent']
compute_optimal_path_flag = config_flag['PRODUCT']['compute_optimal_path']

sub_directory_plots = os.path.join(root_directory_flag, "plots", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_plots, exist_ok=True)
sub_directory_data = os.path.join(root_directory_flag, "data", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_data, exist_ok=True)

check_datetime = pd.to_datetime(datetime_index).strftime('%Y%m%dT%HZ')
check_pattern = os.path.join(sub_directory_data, f"*_DepthAverageData_{check_datetime}.nc")
check_files = glob.glob(check_pattern)
if check_files:
    print(f"Datetime {datetime_index} already processed: {check_files[0]}, skipping task.")
else:
    print(f"Datetime {datetime_index} unprocessed, proceeding with task.")

Datetime 2024-04-26T00:00:00Z unprocessed, proceeding with task.


In [5]:
model_datasets = []

if config_flag['ADVANCED']['reprocess']:
    current_directory = os.getcwd()
    reprocess_path = os.path.join(current_directory, "data/reprocess")

    model_datasets = []
    model_files = glob.glob(os.path.join(reprocess_path, '*.nc'))
    for model_file in model_files:
        depth_average_dataset = xr.open_dataset(model_file)
        model_name = depth_average_dataset.attrs['model_name']
        if model_name == 'RTOFS':
            rtofs_datasets = (None, depth_average_dataset, None)
            model_datasets.append(rtofs_datasets)
        elif model_name == 'CMEMS':
            cmems_datasets = (None, depth_average_dataset, None)
            model_datasets.append(cmems_datasets)
        elif model_name == 'GOFS':
            gofs_datasets = (None, depth_average_dataset, None)
            model_datasets.append(gofs_datasets)
else:
    if enable_rtofs_flag:
        try:
            rtofs = RTOFS()
            rtofs.rtofs_load(config_flag, datetime_index)
            rtofs.rtofs_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
            rtofs_model_data = rtofs.data
            rtofs_depth_average, rtofs_bin_average = interpolate_rtofs(config_flag, sub_directory_data, rtofs_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)
            rtofs_datasets = (rtofs_model_data, rtofs_depth_average, rtofs_bin_average)
            model_datasets.append(rtofs_datasets)
        except Exception as e:
            print(f"Error during RTOFS processing: {e}")
    if enable_cmems_flag:
        try:
            cmems = CMEMS(username='sfricano1', password='GlobalGliders1')
            cmems.cmems_load(config_flag, datetime_index)
            cmems.cmems_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
            cmems_model_data = cmems.data
            cmems_depth_average, cmems_bin_average = interpolate_cmems(config_flag, sub_directory_data, cmems_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)
            cmems_datasets = (cmems_model_data, cmems_depth_average, cmems_bin_average)
            model_datasets.append(cmems_datasets)
        except Exception as e:
            print(f"Error during CMEMS processing: {e}")
    if enable_gofs_flag:
        try:
            gofs = GOFS()
            gofs.gofs_load(config_flag, datetime_index)
            gofs.gofs_save(config_flag, sub_directory_data, save_data=save_model_data_flag)
            gofs_model_data = gofs.data
            gofs_depth_average, gofs_bin_average = interpolate_gofs(config_flag, sub_directory_data, gofs_model_data, chunk=chunk_flag, save_depth_average=save_depth_average_flag, save_bin_average=save_bin_average_flag)
            gofs_datasets = (gofs_model_data, gofs_depth_average, gofs_bin_average)
            model_datasets.append(gofs_datasets)
        except Exception as e:
            print(f"Error during GOFS processing: {e}")


### MODEL DATA: [RTOFS] ###


### INTERPOLATING RTOFS MODEL DATA ###

Start time (UTC): 2024-04-26 13:37:04.134171
End time (UTC): 2024-04-26 13:37:11.170231
Run time: 0:00:07.036060

### MODEL DATA: [CMEMS] ###

INFO - 2024-04-26T13:37:11Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-04-26T13:37:11Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-04-26T13:37:12Z - Service was not specified, the default one was selected: "arco-geo-series"

### INTERPOLATING CMEMS MODEL DATA ###

Start time (UTC): 2024-04-26 13:37:16.119493
End time (UTC): 2024-04-26 13:37:27.565350
Run time: 0:00:11.445857

### MODEL DATA: [GOFS] ###


### INTERPOLATING GOFS MODEL DATA ###

Start time (UTC): 2024-04-26 13:37:28.406073
End time (UTC): 2024-04-26 13:38:06.639499
Run time: 0:00:38.233426


In [6]:
if create_magnitude_plot_flag:
    GGS_plot_magnitude(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        density=density_flag,
        gliders=glider_data_flag,
        show_waypoints=show_waypoints_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
        manual_extent=manual_extent_flag,
        compute_optimal_path=compute_optimal_path_flag
    )
if create_threshold_plot_flag:
    GGS_plot_threshold(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        density=density_flag,
        mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
        gliders=glider_data_flag,
        show_waypoints=show_waypoints_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
        manual_extent=manual_extent_flag,
        compute_optimal_path=compute_optimal_path_flag
    )
if create_advantage_plot_flag:
    GGS_plot_advantage(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        density=density_flag,
        tolerance=tolerance_flag,
        mag1=mag1_flag, mag2=mag2_flag, mag3=mag3_flag, mag4=mag4_flag, mag5=mag5_flag,
        gliders=glider_data_flag,
        show_waypoints=show_waypoints_flag, show_eez=show_eez_flag, show_qc=show_qc_flag,
        manual_extent=manual_extent_flag,
        compute_optimal_path=compute_optimal_path_flag
    )
if create_profiles_plot_flag:
    GGS_plot_profiles(
        config_flag,
        sub_directory_plots,
        datetime_index,
        model_datasets,
        latitude_qc=latitude_qc_flag, longitude_qc=longitude_qc_flag,
        threshold=0.5
    )
if create_gpkg_file_flag:
    GGS_export_gpkg(
        sub_directory_data,
        datetime_index,
        model_datasets
    )


### CREATING PROFILE PLOT ###

Start time (UTC): 2024-04-26 13:38:06.649428
End time (UTC): 2024-04-26 13:38:20.454817
Run time: 0:00:13.805389


---

### GGS Testing

In [7]:
# cmems_depth_average

In [8]:
# latitude_qc = 17.757
# longitude_qc = -67.659

# (y_rtofs_model, x_rtofs_model), _ = calculate_gridpoint(rtofs_model_data, latitude_qc, longitude_qc)
# (y_rtofs_avg, x_rtofs_avg), _ = calculate_gridpoint(rtofs_depth_average, latitude_qc, longitude_qc)
# (y_rtofs_bin, x_rtofs_bin), _ = calculate_gridpoint(rtofs_bin_average, latitude_qc, longitude_qc)

# rtofs_avg_dir = rtofs_depth_average['dir_depth_avg'].isel(y=y_rtofs_avg, x=x_rtofs_avg).values


# print("Diagnostic Information for 'dir_depth_avg':")
# print("Data type:", type(rtofs_avg_dir))
# print("Shape of data:", rtofs_avg_dir.shape)
# print("Data contents:", rtofs_avg_dir)
# if isinstance(rtofs_avg_dir, np.ndarray):
#     print("Array size:", rtofs_avg_dir.size)